In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
!tar xf spark-3.3.0-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-30 20:25:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.75MB/s    in 0.2s    

2022-09-30 20:25:42 (5.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://group-project-data-bucket.s3.amazonaws.com/clean_salary_data.csv"
spark.sparkContext.addFile(url)
salary_data_df = spark.read.csv(SparkFiles.get("clean_salary_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
salary_data_df.show()

+---+-------------------+----------+-----+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+---------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------------+-----+
|_c0|          timestamp|   company|level|               title|totalyearlycompensation|yearsofexperience|yearsatcompany|basesalary|stockgrantvalue|  bonus|rowNumber|Masters_Degree|Bachelors_Degree|Doctorate_Degree|Highschool|Some_College|Race_Asian|Race_White|Race_Two_Or_More|Race_Black|Race_Hispanic|         city|state|
+---+-------------------+----------+-----+--------------------+-----------------------+-----------------+--------------+----------+---------------+-------+---------+--------------+----------------+----------------+----------+------------+----------+----------+----------------+----------+-------------+-------------+-----+
|  0|2017-06-07 11:33:00|    Or

In [6]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgressdb.cuwtf0lwxiav.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "pasgroup3", 
          "driver":"org.postgresql.Driver"}

In [9]:
# Write DataFrame to salary_data table in RDS

salary_data_df.write.jdbc(url=jdbc_url, table='salary_data',mode=mode,properties=config)